# Research Assistant

## Review

We've covered a few major LangGraph themes:

* Memory
* Human-in-the-loop
* Controllability

Now, we'll bring these ideas together to tackle one of AI's most popular applications: research automation. 

Research is often laborious work offloaded to analysts. AI has considerable potential to assist with this.

However, research demands customization: raw LLM outputs are often poorly suited for real-world decision-making workflows. 

Customized, AI-based [research and report generation](https://jxnl.co/writing/2024/06/05/predictions-for-the-future-of-rag/#reports-over-rag) workflows are a promising way to address this.

## Goal

Our goal is to build a lightweight, multi-agent system around chat models that customizes the research process.

`Source Selection` 
* Users can choose any set of input sources for their research.
  
`Planning` 
* Users provide a topic, and the system generates a team of AI analysts, each focusing on one sub-topic.
* `Human-in-the-loop` will be used to refine these sub-topics before research begins.
  
`LLM Utilization`
* Each analyst will conduct in-depth interviews with an expert AI using the selected sources.
* The interview will be a multi-turn conversation to extract detailed insights as shown in the [STORM](https://arxiv.org/abs/2402.14207) paper.
* These interviews will be captured in a using `sub-graphs` with their internal state. 
   
`Research Process`
* Experts will gather information to answer analyst questions in `parallel`.
* And all interviews will be conducted simultaneously through `map-reduce`.

`Output Format` 
* The gathered insights from each interview will be synthesized into a final report.
* We'll use customizable prompts for the report, allowing for a flexible output format. 

![Screenshot 2024-08-26 at 7.26.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb164d61c93d48e604091_research-assistant1.png)

## **Setup**

In [16]:
import os, operator, warnings, getpass
from typing import List, TypedDict, Annotated
from langgraph.graph import MessagesState
from pydantic import BaseModel, Field
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langchain_tavily import TavilySearch
from langchain_community.document_loaders import WikipediaLoader
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Send
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.messages import get_buffer_string
from IPython.display import Image, display, Markdown
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from prompt import *

warnings.filterwarnings("ignore")

In [17]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGSMITH_API_KEY")

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm.invoke("Hello, how are you?").content

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

We'll use [LangSmith](https://docs.langchain.com/langsmith/home) for [tracing](https://docs.langchain.com/langsmith/observability-concepts).

In [6]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "research_assistant"

## **Generate Analysts: Human-In-The-Loop**

Create analysts and review them using human-in-the-loop.

In [9]:
class Analysts(BaseModel):
    affiliation: str = Field(..., description="Primary affiliation of the analyst.")
    name: str = Field(..., description="Name of the analyst.")
    role: str = Field(..., description="Role of the analyst in the context of topic.")
    description: str = Field(..., description="Description of the analyst focus, concerns, and motives.")

    @property
    def personality(self):
        return f"Name: {self.name}, \nRole: {self.role}, \nDescription: {self.description} \nAffiliation: {self.affiliation}"

class Perspectives(BaseModel):
    analyst: List[Analysts] = Field(..., description="comprehensive list of analysts with their roles, descriptions, and affiliations.")

In [12]:
class GenerateAnalystState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[Analysts]

In [14]:
def create_analyst(state: GenerateAnalystState) -> dict:
    """Create an analyst to generate a report."""
    topic = state["topic"]
    max_analysts = state["max_analysts"]
    human_analyst_feedback = state.get("human_analyst_feedback", "")

    # Create a structured LLM
    structured_llm = llm.with_structured_output(Perspectives)
    
    # system prompt
    system_prompt = SystemMessage(
        content=ANALYST_INSTRUCTIONS.format(
            topic=topic,
            max_analysts=max_analysts,
            human_analyst_feedback=human_analyst_feedback,
        ))
    
    # human message
    human_message = HumanMessage(content="Create a set of analysts.")

    # create the analyst
    analysts = structured_llm.invoke([system_prompt, human_message])

    return {"analysts": analysts}

In [18]:
create_analyst({
    "topic": "AI Agents",
    "max_analysts": 2,
})

NameError: name 'ANALYST_INSTRUCTIONS' is not defined